In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import matplotlib
import plot_likert
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the data
data = {}
data['Wave-2020'] = pd.read_csv('../data/processed/scalar/wave_1.csv')
data['Wave-2023'] = pd.read_csv('../data/processed/scalar/wave_5.csv')

In [ ]:
save_fig = True
save_tab = False

## Self-efficacy, response efficacy, and perceived costs

In [ ]:
questions = ['se'] # , 're', 'pc']

mapping = {
    'se': {1.0: "Unable", 2.0: "Somewhat unable", 3.0: "Neither able nor unable", 
           4.0: "Somewhat able", 5.0: "Very able"},
    're': {1: 'Extremely ineffective', 2: 'Somewhat ineffective', 
           3: 'Neither effective nor ineffective', 4: 'Somewhat effective', 
           5: 'Extremely effective'},
    'pc': {1: 'Very cheap', 2: 'Somewhat cheap', 3: 'Neither cheap nor expensive',
           4: 'Somewhat expensive', 5: 'Very expensive'}
}

likert_scale = {
    'se': ["Unable", "Somewhat unable", "Neither able nor unable", 
           "Somewhat able", "Very able"],
    're': ["Extremely ineffective", "Somewhat ineffective", 
           "Neither effective nor ineffective", "Somewhat effective", "Extremely effective"],
    'pc': ["Very cheap", "Somewhat cheap", "Neither cheap nor expensive", 
           "Somewhat expensive", "Very expensive"]
}

colors = ["#ffffff00", "#CA0020", "#F4A582", "#CCCCCC", "#92C5DE", "#0571B0"]
formats = ['png', 'pdf', 'svg', 'eps']

letter = ['a', 'b']
i = 0

for question in questions:

    d = {f'{question}1_raise_ground_floor_level': 'Raise ground floor level',
         f'{question}2_strengthen_foundations': 'Strengthen foundations',
         f'{question}3_reinforce_walls_floor': 'Reinforce walls/floor',
         f'{question}4_raise_electricity_meter': 'Raise electricity meter',
         f'{question}5_install_anti_backflow_valves': 'Install anti-backflow valves',
         f'{question}6_install_pump_drainage': 'Install pump/drainage',
         f'{question}7_fix_water_barriers': 'Fix water barriers'}

    # Remove raise ground floor level

    vars_of_interest = list(d.keys())

    for wave_name, wave_df in data.items():
        df = wave_df[vars_of_interest].copy()
        df.rename(columns=d, inplace=True)       
        df_likert = df.replace(mapping[question])

        # Sort as follows
        sorted_measures = ['Raise electricity meter', 'Install anti-backflow valves', 'Install pump/drainage', 'Fix water barriers', 'Strengthen foundations', 'Reinforce walls/floor', 'Raise ground floor level']

        df_likert = df_likert[sorted_measures]

        # Remove raise ground floor level
        df_likert.pop('Raise ground floor level')
        
        plot_likert.plot_likert(
            df_likert,
            likert_scale[question],
            colors=colors,
            plot_percentage=True,
            bar_labels=True,
            figsize=(9, 5)
        )

        # Change x label
        plt.xlabel('Responses (%)')
        plt.ylabel('Structural measure')
        
        plt.title(f'{wave_name}')
        
        if save_fig:
            for format in formats:
                plt.savefig(
                    f'../figures/fig3{letter[i]}.{format}',
                    dpi=300, bbox_inches='tight'
                )
        i += 1

## Flood frequency

In [ ]:
# There are different categories in wave 2020 and wave 2023
# Map wave 2023 to wave 2020
waves_mapping = {
    'More frequent than once per year': 'More frequent than once per year',
    'Annually': 'Annually',
    'Once in 10 years or 10% chance annually': 'Once in 10 years',
    'Once in 50 years or a 2% chance annually': 'Once in 50 years',
    'Once in 100 years or 1% chance annually': 'Once in 100 years',
    'Once in 200 years or a 0.5% chance annually': 'Once in 200 years',
    'Once in 500 years or a 0.2% chance annually': 'Once in 500 years',
    'Less often than 1 in 500 years': 'Less often than 1 in 500 years',
    'My house is completely safe': 'My house is completely safe',
    "Don't know": "Don't know"
}

data['Wave-2023']['perceived_flood_frequency'].replace(
    waves_mapping, inplace=True)

In [ ]:
vars_of_interest = ['perceived_flood_frequency']
sorted_categories = ['More frequent than once per year', 
                     'Annually', 
                     'Once in 10 years',
                     'Once in 50 years', 
                     'Once in 100 years', 
                     'Once in 200 years',
                     'Once in 500 years', 
                     'Less often than 1 in 500 years',
                     'My house is completely safe',
                     "Don't know"]

# Create single figure
fig, ax = plt.subplots(figsize=(9, 5))

# Plot data for both waves
x = np.arange(len(sorted_categories))
width = 0.35  # Width of bars

for i, (wave_name, wave_df) in enumerate(data.items()):
    df = wave_df[vars_of_interest].copy()
    df.rename(columns={'perceived_flood_frequency': 'Flood frequency'}, inplace=True)

    # Calculate value counts (percentages)
    counts = df['Flood frequency'].value_counts(
        normalize=True).reindex(sorted_categories, fill_value=0) * 100
    
    # Plot bars with offset
    offset = width/2 if i == 0 else -width/2
    bars = ax.barh(x + offset, counts.values, width, 
                   label=wave_name,
                   color='royalblue' if i == 0 else 'lightsteelblue')
    
    # Add value labels
    for bar in bars:
        width_val = bar.get_width()
        ax.text(width_val + 1, bar.get_y() + bar.get_height()/2,
                f'{width_val:.1f}%', va='center', color='black')

# Customize plot
ax.set_ylabel('Percieved flood frequency')
ax.set_xlabel('Responses (%)')
ax.set_yticks(x)
ax.set_yticklabels(sorted_categories)

# Remove spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Add legend outside plot
ax.legend(bbox_to_anchor=(1.1, 1), loc='upper left', borderaxespad=0.)
fig.tight_layout()

FIGURE_NAME = 

if save_fig:
    # Save in three formats
    formats = ['png', 'pdf', 'svg']
    for format in formats:
        plt.savefig(
            f'../figures/results/{FIGURE_NAME}.{format}',
            dpi=300, bbox_inches='tight')

## Flood worry level

In [ ]:
vars_of_interest = ['flood_worry_level']
sorted_categories = ['Not at all worried', 'A little worried', 'Somewhat worried', 'Quite worried', 'Very worried']

# Create figure
fig, ax = plt.subplots(figsize=(6, 4))

# Set width of bars and positions of the bars
width = 0.35
x = np.arange(len(sorted_categories))

# Plot bars for each wave
for i, (wave_name, wave_df) in enumerate(data.items()):
    df = wave_df[vars_of_interest].copy()
    df.rename(columns={'flood_worry_level': 'Flood worry level'}, inplace=True)

    # Calculate value counts (percentages)
    counts = df['Flood worry level'].value_counts(
        normalize=True).reindex(sorted_categories, fill_value=0) * 100
    
    # Plot bars with offset
    offset = width/2 if i == 0 else -width/2
    bars = ax.barh(x + offset, counts.values, width, 
                   label=wave_name,
                   color='royalblue' if i == 0 else 'lightsteelblue')
    
    # Add value labels
    for bar in bars:
        width_val = bar.get_width()
        ax.text(width_val + 1, bar.get_y() + bar.get_height()/2,
                f'{width_val:.0f}%', va='center', color='black')

# Customize plot
ax.set_ylabel('Flood worry level')
ax.set_xlabel('Responses (%)')
ax.set_yticks(x)
ax.set_yticklabels(sorted_categories)

# Remove spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Add legend outside plot
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
fig.tight_layout()

if save_fig:    
    # Save in three formats
    formats = ['png', 'pdf', 'svg', 'eps']
    for format in formats:
        plt.savefig(
            f'../figures/fig2.{format}',
            dpi=300, bbox_inches='tight')   

## Flood experience

In [ ]:
# Set up the plot
fig, ax = plt.subplots(figsize=(6, 4))

# Set width of bars and positions of the bars
width = 0.35
x = np.arange(2)  # 2 categories: No, Yes

for i, (wave_name, wave_df) in enumerate(data.items()):
    counts = wave_df['experienced_flood'].value_counts(
        normalize=True).reindex(['No', 'Yes'], fill_value=0) * 100
    
    # Plot bars with offset
    offset = width/2 if i == 0 else -width/2
    bars = ax.bar(x + offset, counts.values, width,
                  label=wave_name,
                  color='royalblue' if i == 0 else 'lightsteelblue')
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, height + 2,
                f'{int(height)}%', ha='center', va='bottom')

# Customize plot
ax.set_ylabel('Responses (%)')
ax.set_xlabel('Experienced flood')
ax.set_xticks(x)
ax.set_xticklabels(['No', 'Yes'])

# Remove spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Add legend outside plot
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
fig.tight_layout()

FIGURE_NAME =

if save_fig:
    # Save in three formats
    formats = ['png', 'pdf', 'svg']
    for format in formats:
        plt.savefig(
            f'../figures/{FIGURE_NAME}.{format}',
            dpi=300, bbox_inches='tight')

## Responsibility perception

In [ ]:
# Define sorted categories
sorted_categories = ['Completely government', 'Mostly government', 'Equal responsibility',  'Mostly individual', 'Completely individual']

# Create figure
fig, ax = plt.subplots(figsize=(6, 4))

# Set width of bars and positions
width = 0.35
x = np.arange(len(sorted_categories))

# Plot bars for each wave
for i, (wave_name, wave_df) in enumerate(data.items()):
    # Calculate percentages
    counts = wave_df['responsibility_perception'].value_counts()
    percentages = counts.reindex(sorted_categories, fill_value=0) / len(wave_df) * 100
    
    # Plot bars with offset
    offset = width/2 if i == 0 else -width/2
    bars = ax.barh(x + offset, percentages.values, width,
                   label=wave_name,
                   color='#2ecc71' if i == 0 else '#95a5a6')  # Green and gray colors
    
    # Add value labels
    for bar in bars:
        width_val = bar.get_width()
        ax.text(width_val + 1, bar.get_y() + bar.get_height()/2,
                f'{width_val:.0f}%', va='center', color='black')

# Customize plot
ax.set_ylabel('Responsibility perception')
ax.set_xlabel('Reponses (%)')
ax.set_yticks(x)
ax.set_yticklabels(sorted_categories)

# Remove spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Add legend outside plot
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
fig.tight_layout()

if save_fig:
    formats = ['png', 'pdf', 'svg', 'eps']
    for fmt in formats:
        plt.savefig(
            f'../figures/fig4.{fmt}',
            dpi=300, bbox_inches='tight')